In [81]:
from nautilus_trader.config import BacktestVenueConfig
from typing import Optional
from nautilus_trader.core.message import Event
from nautilus_trader.model.data import QuoteTick
from nautilus_trader.model.enums import PriceType
from nautilus_trader.model.enums import PositionSide
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.events import PositionOpened
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.model.objects import Quantity
from nautilus_trader.model.position import Position
from nautilus_trader.trading.strategy import Strategy, StrategyConfig
from nautilus_trader.config import BacktestDataConfig
from nautilus_trader.model.data import QuoteTick
from nautilus_trader.config import BacktestEngineConfig
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.model.objects import Money
from nautilus_trader.model.currencies import BTC
from nautilus_trader.model.currencies import USDT
import os
from nautilus_trader.persistence.loaders import CSVTickDataLoader
from nautilus_trader.persistence.wranglers import BarDataWrangler
from nautilus_trader.test_kit.providers import TestInstrumentProvider
from nautilus_trader.model.data import BarType
from nautilus_trader.model.data import BarSpecification
from nautilus_trader.model.enums import PriceType
from nautilus_trader.model.enums import BarAggregation
from nautilus_trader.model.enums import AggregationSource
from nautilus_trader.model.data import Bar
from nautilus_trader.model.enums import AccountType
from nautilus_trader.model.enums import OmsType
from nautilus_trader.model.identifiers import Venue

In [53]:
btcusdt_instrument = TestInstrumentProvider.btcusdt_binance()

In [55]:
cwd = os.getcwd()
data_path = os.path.join(cwd, "formatted_data/BTCUSDT/1m/1.1.2021-1.1.2022.csv")
df = CSVTickDataLoader.load(file_path=data_path)

In [73]:
bar_spec = BarSpecification(1,BarAggregation.MINUTE, PriceType.LAST)
bar_type=BarType(btcusdt_instrument.id, bar_spec)

In [74]:
wrangler = BarDataWrangler(
    bar_type=bar_type, 
    instrument=btcusdt_instrument,
    )
ticks = wrangler.process(df)

In [77]:
engine_config = BacktestEngineConfig(
        trader_id="BACKTESTER-001",
        logging=LoggingConfig(
            log_level="ERROR",
            log_colors=True,
        ),
    )

In [78]:
engine = BacktestEngine(config=engine_config)


Failed to set global default dispatcher because of error: a global default trace dispatcher has already been set


In [83]:
BINANCE = Venue("BINANCE")
engine.add_venue(
    venue=BINANCE,
    oms_type=OmsType.NETTING,
    account_type=AccountType.CASH,
    base_currency=None,
    starting_balances=[Money(1_000_000.0, USDT), Money(10.0, BTC)],
    
)

In [84]:
engine.add_instrument(btcusdt_instrument)

In [85]:
engine.add_data(ticks)

In [86]:
class VWAPConfig(StrategyConfig):
    InstrumentId: str
    fast_period: int = 12
    slow_period: int = 26
    trade_size: int = 100
    entry_threshold: float = 0.0001